In [0]:
%pip install -r ../../requirements.txt
%restart_python

In [0]:
import yfinance as yf
from datetime import datetime, date
import os
import time
from pyspark.sql.functions import lit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, TimestampType
import yaml
from pyspark.sql.functions import to_date
from pyspark.sql import types as T
# --- DODATKOWY IMPORT DLA DELTA LAKE ---
from delta.tables import DeltaTable
from pyspark.sql.functions import col

try:
    # dbutils.widgets.get() jest standardową metodą odczytu parametrów Joba
    ENV = dbutils.widgets.get("env_name") 
    print(f"Środowisko skonfigurowane przez Job (ENV): {ENV}")
except Exception:
    # Obsługa błędu - gdy uruchamiasz notebook interaktywnie lub ręcznie (spoza Joba),
    # dbutils.widgets.get() może zgłosić błąd, bo widgety nie są zainicjowane.
    # W takim przypadku ustawiamy wartość domyślną (np. 'TEST')
    ENV = 'TEST' 
    print(f"Uruchomienie interaktywne/ręczne. Użycie domyślnego ENV: {ENV}")


try:
    # W Databricks, ścieżka do pliku YAML może wymagać dostosowania,
    # jeśli nie jest on w tym samym katalogu co notatnik.
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("BŁĄD: Plik 'config.yaml' nie został znaleziony! Sprawdź ścieżkę.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Nie znaleziono konfiguracji dla środowiska: {ENV} w pliku YAML.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']
sql_table = CFG['sql_table_name']
BRONZE_TABLE_PATH = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_bronze_data/" 


In [0]:
# --- KROK 1: Tworzenie df_tickets z tabeli Databricks SQL ---
try:
    sql_query = f"SELECT Ticket, company_name FROM {sql_table}"
    spark_df_tickets = spark.sql(sql_query)
    df_tickets_list = spark_df_tickets.collect()
    company_info_dict = {row['Ticket']: row['company_name'] for row in df_tickets_list}
    gpw_ticket_list = list(company_info_dict.keys())
    print("Pomyślnie utworzono Spark DataFrame z tabeli SQL.")
    print("Akcje do przetworzenia:", gpw_ticket_list)
except Exception as e:
    print(f"Błąd odczytu z tabeli SQL: {e}.")
    gpw_ticket_list = []
    company_info_dict = {}


In [0]:
# Utworzenie katalogu bazowego, jeśli nie istnieje
try:
    dbutils.fs.mkdirs(BRONZE_TABLE_PATH)
    print(f"Pomyślnie utworzono katalog dla JEDNEJ TABELI: {BRONZE_TABLE_PATH}")
except Exception as e:
    print(f"Katalog już istnieje lub wystąpił błąd podczas jego tworzenia: {e}")

# Ustawienia zakresu dat dla yfinance
end_date = date.today()

if not gpw_ticket_list:
    print("Brak danych firm do przetworzenia. Używanie danych zastępczych dla demonstracji.")




In [0]:
# Helper function to check if a path is a Delta Table
def is_delta_table(path):
    """Sprawdza, czy podana ścieżka zawiera folder _delta_log (czyli jest Delta Table)."""
    try:
        # dbutils.fs.ls zgłosi błąd, jeśli ścieżka nie istnieje.
        dbutils.fs.ls(os.path.join(path, "_delta_log")) 
        return True
    except Exception:
        return False
is_full_table_existing = is_delta_table(BRONZE_TABLE_PATH)

print("\nRozpoczynanie pobierania danych z yfinance i zapisywanie do Delta Lake...")

for t in gpw_ticket_list:
    yfinance_ticker = f"{t}.WA"
    print(f"Przetwarzanie akcji: {yfinance_ticker}")

    # --- LOGIKA POBIERANIA DANYCH ---
    start_date_for_update = None
    last_download_date = None

    if is_full_table_existing:
        try:
            # ODZCYT Z JEDNEJ DUŻEJ TABELI: Odczytujemy maksymalną datę DLA TEGO KONKRETNEGO TICKERA
            # Używamy predykatu filtrującego, który zadziała jako pruning na partycjach
            max_date_df = spark.read.format("delta").load(BRONZE_TABLE_PATH) \
                            .filter(F.col("Ticket") == t) \
                            .select(F.max(F.col("Date").cast("date")).alias("max_date"))

            last_download_date = max_date_df.collect()[0]['max_date']

            if last_download_date and last_download_date < date.today():
                # Ustawiamy datę początkową na dzień po ostatnio pobranym
                start_date_for_update = (pd.to_datetime(last_download_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
                print(f"Dane dla '{yfinance_ticker}' wymagają aktualizacji od daty: {start_date_for_update}.")
            elif last_download_date:
                print(f"Dane dla '{yfinance_ticker}' są aktualne na dzień {last_download_date}. Pomijanie.")
                time.sleep(1) # Unikaj przeciążania API
                continue
            else:
                # Jeśli last_download_date jest None (ticker nie istnieje w tabeli lub tabela pusta)
                print(f"Ticker '{yfinance_ticker}' nie został znaleziony w głównej tabeli. Pełne pobranie.")
                
        except Exception as e:
            print(f"Wystąpił błąd odczytu/filtra na głównej Delta Table dla '{yfinance_ticker}': {e}. Pełne ponowne pobranie.")
            # Jeśli jest błąd, pozostawiamy is_full_table_existing na True, 
            # ale proceedujemy do pełnego pobierania dla bezpieczeństwa.
    
            
    # Ustawienia daty początkowej dla yfinance
    if not is_full_table_existing or not last_download_date:
        # Pełne pobieranie od maksymalnej dostępnej daty (gdy tabela nie istniala lub ticker nie istniał)
        data = yf.download(yfinance_ticker, period='max', end=end_date)
        print(f"Rozpoczęcie pełnego pobierania dla '{yfinance_ticker}'.")
    elif start_date_for_update:
        # Pobieranie tylko nowych danych (gdy tabela istniala i ticker istnial)
        data = yf.download(yfinance_ticker, start=start_date_for_update, end=end_date)
        print(f"Pobieranie danych przyrostowych dla '{yfinance_ticker}'.")
    else:
        # Dane aktualne (obsłużone przez 'continue' w bloku powyżej, ale zabezpieczenie)
        time.sleep(1)
        continue

    # --- LOGIKA ZAPISU DO DELTA LAKE ---
    if not data.empty:
        # Przygotowanie danych do zapisu Spark DataFrame
        df_new_data = data.reset_index()
        # Obsługa MultiIndex po pobraniu (jeśli wystąpi)
        if isinstance(df_new_data.columns, pd.MultiIndex):
            df_new_data.columns = df_new_data.columns.map(lambda x: x[0])
        # Konwersja do Spark DataFrame
        spark_new_data_df = spark.createDataFrame(df_new_data)

        # Czyszczenie i dodawanie kolumn
        spark_new_data_df = spark_new_data_df.withColumn("Date", col("Date").cast("date")) \
                                              .withColumn("Ticket", lit(t)) \
                                              .withColumn("company_name", lit(company_info_dict.get(t, 'N/A')))
        # Upewnienie się, że DF ma wymagane kolumny
        cols_to_select = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticket', 'company_name']
        spark_new_data_df = spark_new_data_df.select([F.col(c) for c in cols_to_select if c in spark_new_data_df.columns])

        # A) Jeśli tabela NIE ISTNIEJE (po pierwszym przebiegu), tworzymy nową jako Delta Lake
        if not is_full_table_existing:
            try:
                # Tworzymy pierwszą wersję JEDNEJ GŁÓWNEJ TABELI Delta (partycjonowanej przez Ticket)
                spark_new_data_df.write.format("delta").mode("overwrite").partitionBy("Ticket").save(BRONZE_TABLE_PATH)
                print(f"Pobrano i zapisano **pierwszą pełną** partię do JEDNEJ TABELI Delta.")
                is_full_table_existing = True # Oznaczamy, że tabela już istnieje
            except Exception as e:
                print(f"BŁĄD ZAPISU PEŁNEJ DELTA TABLE dla '{yfinance_ticker}': {e}")
        # B) Jeśli tabela ISTNIEJE, używamy MERGE
        else:
            try:
                # Używamy MERGE INTO do transakcyjnej aktualizacji/wstawiania w JEDNEJ GŁÓWNEJ TABELI
                deltaTable = DeltaTable.forPath(spark, BRONZE_TABLE_PATH)
                
                deltaTable.alias("target") \
                    .merge(
                        source=spark_new_data_df.alias("source"),
                        # Łączymy na podstawie 'Date' i 'Ticket' (kolumny klucza biznesowego)
                        condition="target.Date = source.Date AND target.Ticket = source.Ticket"
                    ) \
                    .whenMatchedUpdateAll() \
                    .whenNotMatchedInsertAll() \
                    .execute()
                
                print(f"Zaktualizowano dane dla '{yfinance_ticker}' za pomocą **MERGE INTO** w JEDNEJ TABELI Delta Lake.")
            except Exception as e:
                print(f"BŁĄD MERGE INTO DELTA LAKE dla '{yfinance_ticker}': {e}.")
                
    else:
        print(f"Brak nowych danych do pobrania dla '{yfinance_ticker}'.")
        
    time.sleep(1) # Unikaj przeciążania API
    
print("\n--- ZAKOŃCZONO PROCES AKTUALIZACJI/WSTAWIANIA DO JEDNEJ GŁÓWNEJ TABELI DELTA LAKE ---")
